In [30]:
#import modules
import json
from binance.client import Client
import pandas as pd
from envs import env
from dotenv import load_dotenv

In [31]:
#import classes from ./ folder
import timescaledbAccess

In [32]:
#read env vars
load_dotenv()

True

In [33]:
timescale = timescaledbAccess.timescaleAccess()
try:
    liveVolume=env("liveVolume")
    apiSecret=env('apiSecret')
    apiKey=env('apiKey')
except KeyError:
    print("No env variables set.")
    sys.exit(1)
#connect to binance to get current balance
client = Client(apiKey, apiSecret, {'timeout':600})

In [34]:
if len(client.get_open_orders()) == 0:
    print("YY")

YY


In [35]:
client.get_asset_balance(asset='BNB')['free']

'0.00000000'

In [36]:
sql = ("SELECT id, askprice, managedassets" +
    " FROM table001 WHERE" +
    " resultpercent IS NULL " +
    " AND takeprofit IS NOT NULL;")
bA = pd.DataFrame(timescale.sqlQuery(sql))
bA = bA.apply(pd.to_numeric, errors='coerce')

In [37]:
for i in range(1,3):
    print(i)

1
2


In [43]:
sql = ("select symbol from table001 where id = '1';")
symbol = pd.DataFrame(timescale.sqlQuery(sql))
sql = ("select max(id) from table001 where symbol = '" + str(symbol[0][0]) + "';")
maxId = pd.DataFrame(timescale.sqlQuery(sql))

In [44]:
print(maxId[0][0])

12199
